In [10]:
import time
import jwt   # pyjwt (small dependency, but not Google)
import requests
import json
import base64
import requests
from email.mime.text import MIMEText

In [5]:
private_key = json.load(open("somkey.json"))['private_key']

In [18]:

SERVICE_ACCOUNT_EMAIL = "valju-email-server-test@makro-website.iam.gserviceaccount.com"
IMPERSONATE_USER = "noreply@valju.is"
PRIVATE_KEY = private_key

now = int(time.time())

claims = {
    "iss": SERVICE_ACCOUNT_EMAIL,
    "sub": IMPERSONATE_USER,
    "aud": "https://oauth2.googleapis.com/token",
    "iat": now,
    "exp": now + 3600,
    "scope": "https://www.googleapis.com/auth/gmail.send",
}

jwt_token = jwt.encode(claims, PRIVATE_KEY, algorithm="RS256")

resp = requests.post(
    "https://oauth2.googleapis.com/token",
    data={
        "grant_type": "urn:ietf:params:oauth:grant-type:jwt-bearer",
        "assertion": jwt_token,
    },
)

ACCESS_TOKEN = resp.json()["access_token"]

In [19]:


def send_email(to: str, subject: str, body: str):
    # Build RFC 2822 message
    msg = MIMEText(body, _charset="utf-8")
    msg["To"] = to
    msg["Subject"] = subject

    # Base64url encode
    raw = base64.urlsafe_b64encode(msg.as_bytes()).decode("utf-8")

    # Gmail API request
    resp = requests.post(
        "https://gmail.googleapis.com/gmail/v1/users/me/messages/send",
        headers={
            "Authorization": f"Bearer {ACCESS_TOKEN}",
            "Content-Type": "application/json",
        },
        json={"raw": raw},
        timeout=10,
    )

    # Raise on error for easier debugging
    #resp.raise_for_status()
    return resp.json()

In [20]:
send_email('palmar@valju.is', "hello", 'Hello Hello')

{'id': '19be0d6da837542c',
 'threadId': '19be0d6da837542c',
 'labelIds': ['SENT']}